Start jupyter-lab

```bash
jupyter-lab --notebook-dir=$HOME/gits/gerashegalov/rapids-shell/src/jupyter
```
or simply open in VS Code with Jupyter extension

In [ ]:
import findspark
import os

In [ ]:
os.environ['SPARK_LOCAL_IP'] = '127.0.0.1'
os.environ['TZ']='UTC'
home_dir = os.environ['HOME']
work_dir = f"{home_dir}/jupyter_run_dir"
spark_home = f"{home_dir}/dist/spark-3.4.0-bin-hadoop3"
cores_per_exec = 1
spark_master = f"local[{cores_per_exec}]"
rapids_version='23.08.0-SNAPSHOT'
findspark.init(spark_home=spark_home)
findspark.add_jars(f"{home_dir}/gits/NVIDIA/spark-rapids/dist/target/rapids-4-spark_2.12-{rapids_version}-cuda11.jar")


In [ ]:
import pyspark

spark = pyspark.sql.SparkSession.builder \
    .appName('RAPIDS PySpark Notebook') \
    .master(spark_master) \
    .config('spark.plugins', 'com.nvidia.spark.SQLPlugin') \
    .config('spark.rapids.sql.explain', 'ALL') \
    .config('spark.rapids.sql.expression.JsonToStructs', True) \
    .getOrCreate()

In [ ]:
import sys
sys.path.append(f"{home_dir}/gits/NVIDIA/spark-rapids/integration_tests/src/main/python")
import spark_init_internal
setattr(spark_init_internal, '_spark', spark)
from data_gen import *

In [ ]:
df = unary_op_df(spark, StringGen('', nullable=True))

In [ ]:
schema=[
    StructType([StructField("a", StringType())]),
    StructType([
        StructField("a", StringType()), 
        StructField("b", IntegerType())
    ])
]

In [ ]:
simple_schema = list(map(lambda s: s.simpleString(), schema))

In [ ]:
simple_schema = ['struct<a:struct<a:string>>', 'struct<a:string,b:int>']

In [ ]:
df.select(pyspark.sql.functions.from_json('a', simple_schema[0])).collect()

In [ ]:
spark.createDataFrame([('gera',),], 'a:string')